在 restful 理念大行其道的今天，很多网站都提供了类 restful api。当客户调用这些 API 时，服务器会返回一个能表示资源状态的字符串，可以用纯文本，JSON，XML，但通常采用的都是 JSON。因为 JSON 的表达能力强，可以嵌套，并且相比 XML 的冗余来说，非常精简。如我们前面用过的http://httpbin.org/get ，还有 https://api.github.com/ 等。可以看到，这些网站返回的都是 JSON 格式的字符串。
$\alpha = \beta$
$$ \Omega :=\{ x | 1\le x\le 3 \}$$

    知识点1
    知识点2
    知识点3
    知识点4

通常前端利用 Javascript 发起 XMLHttpRequest 请求，然后将得到的 JSON 字符串转换成一个对象。最后从这个对象中提取出想要的信息，并展示在页面上。数据爬取也类似，只是我们用 requests 模拟 XMLHttpRequest 发起请求，然后将得到的 JSON 字符串转换为 Python 对象，最后再从对象中提取数据。相比从 HTML 中提取数据，从 JSON 中提取数据更好一点，因为 HTML 的模板可能经常变动，这样就可能会导致提取失败。但 api 通常是很少变化的，并且服务器返回的 JSON 字符串的格式通常是很规整的，并且很少变化。并且 JSON 中通常只包含最有价值的信息，会占用更少的网络带宽，解析速度也会更快。所以推荐在能从 JSON 中提取数据的话就从 JSON 中提取（有时，前段 HTML 只是一个框架，不包含数据，数据都是通过XMLHttpRequest异步加载的，这种情况下，只能从 JSON 中提取）。


In [2]:
import requests
import urllib

url = 'http://httpbin.org/get'
response = requests.get(url, timeout=5)
print (response.text)


{
  "args": {}, 
  "headers": {
    "Accept": "*/*", 
    "Accept-Encoding": "gzip, deflate", 
    "Host": "httpbin.org", 
    "User-Agent": "python-requests/2.23.0", 
    "X-Amzn-Trace-Id": "Root=1-5e69ab32-7acc1a1a411c72f00bb552ce"
  }, 
  "origin": "110.53.149.207", 
  "url": "http://httpbin.org/get"
}





可以看到，我们得到的响应是 JSON 格式的字符串。



## 解析 JSON

将 JSON 字符串转换成 Python 对象也很简单。通常我们会选择用 Python 的原生库 json 来解析和转换 JSON字符串：

In [3]:
import json
json_data = json.loads(response.text)
# response.json() 是 json.loads(response.text) 的快捷方式
print (type(json_data))
print (json_data)

<class 'dict'>
{'args': {}, 'headers': {'Accept': '*/*', 'Accept-Encoding': 'gzip, deflate', 'Host': 'httpbin.org', 'User-Agent': 'python-requests/2.23.0', 'X-Amzn-Trace-Id': 'Root=1-5e69ab32-7acc1a1a411c72f00bb552ce'}, 'origin': '110.53.149.207', 'url': 'http://httpbin.org/get'}


可以看到，用 json.loads 可以很容易地将 JSON 字符串转换成一个 dict 对象，并且这个方法内部采用 unicode 字符串（推荐这样做，内部使用 unicode 字符串，等需要输出的时候再调用 encode 进行编码）。

## 数据提取

解析 JSON 字符串之后得到 dict 对象，也就是说提取数据其实就是字典操作，当然非常简单了： 

In [4]:
print (json_data['origin'])
print (json_data.get('headers', {}).get('User-Agent', None))
print (json_data.get('no-such-key', {}).get('User-Agent', None))

110.53.149.207
python-requests/2.23.0
None


上面的代码提取了'origin'，'User-Agent'。都是很简单的字典操作。需要注意的是，有些服务为了尽量节省带宽，在生成 JSON 字符串时，会忽略那些内容为空的 key，如上面的 args。所以，为了程序的鲁棒性，推荐使用 get 来从 dict 中提取数据，如最后一行，即使 dict 中不存在要找的 key，程序也不会出错。



上面的练习是一个综合性比较强的练习，涉及到了网络请求，参数编解码，JSON 转换，dict 解码等内容。需要程序员对整个网络的交互过程和 Python 中字符串的编码有比较清晰的认识。如果你很顺利地完成了这个练习，恭喜你！这说明前面的内容你学的不错。遇到困难也没关系，随着不断的练习，相信你会理解的越来越清楚，用的越来越熟练。这里大致解释一下整个网络的交互过程：

-    构造 profile dict。
-    将 profile 转换为字符串，并进行编码，然后添加到 URL 的末尾作为查询参数。
-    发起请求，得到 response。
-    从 response 返回的 json 中提取出我们发送的查询参数的值，得到一个字符串。
-    将这个字符串进行解码，得到 JSON 字符串，然后再转换成 dict，得到 new_profile。
-    比较 profile 和 new_profile。

需要注意的是，程序的倒数第二行用于将 dict 中的 unicode 字符串，转换为字节数组表示的字符串，巧妙地结合了 json.dumps 方法和 eval 方法。当然你也可以把 dict 中的每个 key，每个 value 逐一读出来，再加以转换。但当 value 也是比较复杂的对象，如 list，dict 时，这种方法就显得比较繁琐了。

好了，我们又完成了一章，学会了数据的提取。数据提取之后当然要存储，我们将在下一章讲解。